# Data cleaning in PySpark

## Import library

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_unixtime, col, isnan, when, count, to_date, unix_timestamp, date_format, to_timestamp, lit

from pyspark.sql.functions import round as spark_round
from pyspark.sql.functions import split as spark_split

import pandas as pd
from tqdm import  tqdm

import os, sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

## Load Spark session

In [2]:
spark = SparkSession.builder.master("local[*]").config("spark.driver.memory", "15g").config("spark.driver.maxResultSize", "8g").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/01 15:36:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load dataset

In [3]:
assignment = spark.read.csv('../dataset/raw/assignments.tsv', sep='\t',header=True)
assignment.show(5)

print('column')
print(assignment.columns)

+---------------------------+-------------------+--------------------+----------------+---------------------+---------------------+-----------------+----------------------+---------------+-----------------+----------------+---------+--------------------+
|assignment_gs_correct_count|assignment_gs_count|       assignment_id|assignment_price|assignment_project_id|assignment_start_time|assignment_status|assignment_submit_time|assignment_type|  device_category|microtasks_count|os_family|             user_id|
+---------------------------+-------------------+--------------------+----------------+---------------------+---------------------+-----------------+----------------------+---------------+-----------------+----------------+---------+--------------------+
|                         13|                 15|fe52c68a-48ec-483...|             0.0|                   14|           1538145644|         APPROVED|            1538145992|          train|PERSONAL_COMPUTER|              15|  WINDOWS|28

In [4]:
proj = spark.read.csv('../dataset/raw/projects-data.tsv', sep='\t',header=True)
proj.show(5)

print('column')
print(proj.columns)

24/08/01 15:36:08 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------------+-----------------+------------------+-------------------------------+--------------------------+------------------------+---------------------------+-------------------------+---------------------------+----------------------+------------------+-----------------+---------------------------------+-----------------------+---------------+------------------------+---------------------------------+------------------------+-------------------------+--------------------------+-----------------+----------+----------------------+----------------------------+-----------------------+-----------------------------+-----------------------+-------------------+
|project_creation_timestamp|project_has_audio|project_has_button|project_has_buttonClicked_input|project_has_checkbox_input|project_has_externalHtml|project_has_fileAudio_input|project_has_fileImg_input|project_has_fileVideo_input|project_has_file_input|project_has_iframe|project_has_image|project_has_imageAnnotation

# Clean dataset

Follow the steps below
1. remove column 'assignment_id' and 'user_id' from *assignment* dataframe
2. remove column 'project_creation_timestamp' from *project* dataframe
3. remove duplicate rows
4. join *assignment* and *project* dataframe by project id
5. remove project id column
6. remove duplicate rows
7. check missing values
8. check columns that should be removed
9. convert timestamp to date and time

In [5]:
## step 1-2
assignment = assignment.drop('assignment_id', 'user_id')
proj = proj.drop('project_creation_timestamp')

In [6]:
## step 3
assignment = assignment.drop_duplicates()
proj = proj.drop_duplicates()

In [7]:
## step 4
join_df = assignment.join(proj, assignment.assignment_project_id == proj.project_id, how = 'inner')

print('total rows in assignment df:', assignment.count())
print('total rows in proj df:', proj.count())
print('total joined rows in join_df:', join_df.count())

total rows in assignment df: 18507067
total rows in proj df: 767


total joined rows in join_df: 18507067


In [8]:
## step 5
join_df = join_df.drop('project_id', 'assignment_project_id')

In [9]:
## step 6
join_df = join_df.drop_duplicates()
print('total joined rows in join_df:', join_df.count())

total joined rows in join_df: 18507066


In step 7, 
- there are 28 projects that the value of *project_instruction_FK* is null
- there are 18 projects that the value of *project_instruction_language*, *project_instruction_len*, *project_instruction_wordCount* is null

This results in 117081 total records (0.63% of 18507066) that contain null value.
Thus, these records will be dropped out

In [10]:
## step 7
join_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in join_df.columns]).show()

+---------------------------+-------------------+----------------+---------------------+-----------------+----------------------+---------------+---------------+----------------+---------+-----------------+------------------+-------------------------------+--------------------------+------------------------+---------------------------+-------------------------+---------------------------+----------------------+------------------+-----------------+---------------------------------+-----------------------+---------------+------------------------+---------------------------------+------------------------+-------------------------+--------------------------+-----------------+----------------------+----------------------------+-----------------------+-----------------------------+-----------------------+-------------------+
|assignment_gs_correct_count|assignment_gs_count|assignment_price|assignment_start_time|assignment_status|assignment_submit_time|assignment_type|device_category|microtas

In [11]:
proj.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in proj.columns]).show()

+-----------------+------------------+-------------------------------+--------------------------+------------------------+---------------------------+-------------------------+---------------------------+----------------------+------------------+-----------------+---------------------------------+-----------------------+---------------+------------------------+---------------------------------+------------------------+-------------------------+--------------------------+-----------------+----------+----------------------+----------------------------+-----------------------+-----------------------------+-----------------------+-------------------+
|project_has_audio|project_has_button|project_has_buttonClicked_input|project_has_checkbox_input|project_has_externalHtml|project_has_fileAudio_input|project_has_fileImg_input|project_has_fileVideo_input|project_has_file_input|project_has_iframe|project_has_image|project_has_imageAnnotation_input|project_has_radio_input|project_has_sbs|project

In [12]:
join_df = join_df.dropna()

In [14]:
## step 8

for col in ['project_has_audio', 'project_has_button', 'project_has_buttonClicked_input', 'project_has_checkbox_input', 'project_has_externalHtml', 'project_has_fileAudio_input', 'project_has_fileImg_input', 'project_has_fileVideo_input', 'project_has_file_input', 'project_has_iframe', 'project_has_image', 'project_has_imageAnnotation_input', 'project_has_radio_input', 'project_has_sbs', 'project_has_select_input', 'project_has_sourcesRecorder_input', 'project_has_string_input', 'project_has_suggest_input', 'project_has_textarea_input', 'project_has_video', 'project_instruction_language']:
    print('unique value of column {}: {}'.format(col, join_df.select(col).distinct().count()))

unique value of column project_has_audio: 5


unique value of column project_has_button: 16


unique value of column project_has_buttonClicked_input: 1


unique value of column project_has_checkbox_input: 19


unique value of column project_has_externalHtml: 2


...

unique value of column project_has_string_input: 12


unique value of column project_has_suggest_input: 4


unique value of column project_has_textarea_input: 8


unique value of column project_has_video: 3


unique value of column project_instruction_language: 2


In [17]:
join_df.select('project_has_fileAudio_input').distinct().show()

+---------------------------+
|project_has_fileAudio_input|
+---------------------------+
|                          0|
|                          1|
+---------------------------+



In [19]:
### drop columns that have only 1 unique value

col_to_drop = ['project_has_buttonClicked_input', 'project_has_imageAnnotation_input', 'project_has_fileVideo_input']

for col in col_to_drop:
    join_df = join_df.drop(col)


In [35]:
## step 9

join_df = join_df.withColumn('assignment_start_time', from_unixtime(col('assignment_start_time')))
join_df = join_df.withColumn('assignment_submit_time', from_unixtime(col('assignment_submit_time')))

### convert to human-readable format
join_df = join_df.withColumn('assignment_start_time', to_timestamp(col('assignment_start_time'), 'yyyy-MM-dd HH:mm:ss'))
join_df = join_df.withColumn('assignment_submit_time', to_timestamp(col('assignment_submit_time'), 'yyyy-MM-dd HH:mm:ss'))

join_df = join_df.withColumn('completion-time-in-minutes', spark_round((unix_timestamp('assignment_submit_time') - unix_timestamp('assignment_start_time'))/60,2))

join_df.show(5)

+---------------------------+-------------------+----------------+---------------------+-----------------+----------------------+---------------+-----------------+----------------+---------+-----------------+------------------+--------------------------+------------------------+---------------------------+-------------------------+----------------------+------------------+-----------------+-----------------------+---------------+------------------------+---------------------------------+------------------------+-------------------------+--------------------------+-----------------+----------------------+----------------------------+-----------------------+-----------------------------+-----------------------+-------------------+--------------------------+
|assignment_gs_correct_count|assignment_gs_count|assignment_price|assignment_start_time|assignment_status|assignment_submit_time|assignment_type|  device_category|microtasks_count|os_family|project_has_audio|project_has_button|project

In [36]:
## split date and time into 2 columns

join_df = join_df.withColumn('assignment_start_date', date_format('assignment_start_time', 'yyyy-M-d'))
join_df = join_df.withColumn('assignment_start_time', date_format('assignment_start_time', 'h:m:s'))
join_df = join_df.withColumn('assignment_submit_date', date_format('assignment_submit_time', 'yyyy-M-d'))
join_df = join_df.withColumn('assignment_submit_time', date_format('assignment_submit_time', 'h:m:s'))

join_df.show(5)

+---------------------------+-------------------+----------------+---------------------+-----------------+----------------------+---------------+-----------------+----------------+---------+-----------------+------------------+--------------------------+------------------------+---------------------------+-------------------------+----------------------+------------------+-----------------+-----------------------+---------------+------------------------+---------------------------------+------------------------+-------------------------+--------------------------+-----------------+----------------------+----------------------------+-----------------------+-----------------------------+-----------------------+-------------------+--------------------------+---------------------+----------------------+
|assignment_gs_correct_count|assignment_gs_count|assignment_price|assignment_start_time|assignment_status|assignment_submit_time|assignment_type|  device_category|microtasks_count|os_family

In [37]:
## split date into 3 columns (year, month, day)
## split time into 2 columns (hour and minute)

split_start_date = spark_split(join_df['assignment_start_date'], '-')
split_start_time = spark_split(join_df['assignment_start_time'], ':')

join_df = join_df.withColumn('assignment_start_year', split_start_date.getItem(0))
join_df = join_df.withColumn('assignment_start_month', split_start_date.getItem(1))
join_df = join_df.withColumn('assignment_start_day', split_start_date.getItem(2))

join_df = join_df.withColumn('assignment_start_hour', split_start_time.getItem(0))
join_df = join_df.withColumn('assignment_start_minute', split_start_time.getItem(1))



split_submit_date = spark_split(join_df['assignment_submit_date'], '-')
split_submit_time = spark_split(join_df['assignment_submit_time'], ':')

join_df = join_df.withColumn('assignment_submit_year', split_submit_date.getItem(0))
join_df = join_df.withColumn('assignment_submit_month', split_submit_date.getItem(1))
join_df = join_df.withColumn('assignment_submit_day', split_submit_date.getItem(2))

join_df = join_df.withColumn('assignment_submit_hour', split_submit_time.getItem(0))
join_df = join_df.withColumn('assignment_submit_minute', split_submit_time.getItem(1))

join_df.show(5)

+---------------------------+-------------------+----------------+---------------------+-----------------+----------------------+---------------+-----------------+----------------+---------+-----------------+------------------+--------------------------+------------------------+---------------------------+-------------------------+----------------------+------------------+-----------------+-----------------------+---------------+------------------------+---------------------------------+------------------------+-------------------------+--------------------------+-----------------+----------------------+----------------------------+-----------------------+-----------------------------+-----------------------+-------------------+--------------------------+---------------------+----------------------+---------------------+----------------------+--------------------+---------------------+-----------------------+----------------------+-----------------------+---------------------+---------

In [40]:
join_df.select('project_has_fileImg_input').distinct().show()

+-------------------------+
|project_has_fileImg_input|
+-------------------------+
|                        0|
|                        1|
|                        4|
+-------------------------+



In [42]:
join_df.select('project_has_suggest_input').distinct().show()

+-------------------------+
|project_has_suggest_input|
+-------------------------+
|                        3|
|                        0|
|                       32|
|                        1|
+-------------------------+



# Save the cleaned dataset for further steps

In [43]:
join_df.write.csv('../dataset/cleaned/cleaned_dataset_from_spark')

In [44]:
## pyspark saves multiple csv files in a specified directory
## to get a single csv file, pandas is used to combine these csv files

dir = '../dataset/cleaned/cleaned_dataset_from_spark/'

all_files = os.listdir(dir)

all_files = [f for f in all_files if f.startswith('part')]

print(all_files)

all_df = []

for f in tqdm(all_files):
    sub_df = pd.read_csv(os.path.join(dir, f), header=None)
    sub_df.columns = join_df.columns
    all_df.append(sub_df)

final_df = pd.concat(all_df)

final_df.to_csv('../dataset/cleaned/cleaned_dataset.csv', index=False)

['part-00007-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00028-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00024-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00010-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00033-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00029-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00006-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00025-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00011-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00032-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00026-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00009-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00005-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00031-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00012-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00008-cfe64d09-75a9-4161-8021-8b0f4bd06a95-c000.csv', 'part-00027-cfe64d09-75a9-4161-8021-8b0

100%|██████████| 40/40 [01:30<00:00,  2.27s/it]
